In [1]:
import SimpleITK as sitk
import numpy as np 
import matplotlib.pyplot as plt
from pathlib import Path
from mrio import dicom_reader,nifti_reader
from models.roi import create_rois, ROI  # Import create_rois and ROI class
from fitting.relaxation_fitting import RelaxationFittingModel
from process import transform
from fitting.analysis import compute_roi_statistics
from visualization.mapplot import (
    show_slice,
    show_mask,
    show_parameter_map_with_rois,
    show_rois_with_mean,
)

In [2]:
input_dicom_folder_1 = '/home/gizmoo/Desktop/C01/CHockey_01_T0_l/DICOM_translated/MRTC-Studie_Hockey_K01_GAPF81827/20210907_1825/6_t2_de3d_we_cor_11121'
input_mask_file= '/home/gizmoo/Desktop/C01/masks/Cor_Maske_C01_T0_l_04092024_1321.nii.gz'
input_dicom_folder_2 = '/home/gizmoo/Desktop/C01/CHockey_01_T0_l/DICOM_translated/MRTC-Studie_Hockey_K01_GAPF81827/20210907_1825/7_T2-star_map_3D_cor_12541'
out_nii_file = '/home/gizmoo/new_mask.nii.gz'

In [3]:
# Convert the paths to Path objects
transform(
        input_dicom_folder_1=Path(input_dicom_folder_1),
        input_mask_file=Path(input_mask_file),
        input_dicom_folder_2=Path(input_dicom_folder_2),
        out_nii_file=Path(out_nii_file),
        reverse=False  # Set to True if you need to reverse the image order
    )
